In [1]:
words = open('names.txt', 'r').read().splitlines()
words[:2]

['emma', 'olivia']

In [2]:
bi = {}
for word in words[:1]:
    w = ['.', '.'] + list(word) + ['.']

    for ch1, ch2, ch3 in zip(w, w[1:], w[2:]):
        print(f"{ch1} {ch2} -> {ch3}")


. . -> e
. e -> m
e m -> m
m m -> a
m a -> .


In [40]:
import torch
N = torch.zeros((27, 27, 27), dtype=torch.int32)
N.dtype

torch.int32

## tigram counting method

In [4]:
chars = ['.'] + sorted(set(''.join(words)))
stoi = {k:i for i, k in enumerate(chars)}
itos = {v:k for k, v in stoi.items()}

In [41]:

for word in words:
    w = ['.', '.'] + list(word) + ['.']

    for ch1, ch2, ch3 in zip(w, w[1:], w[2:]):
        idx = (stoi[ch1], stoi[ch2], stoi[ch3])
        N[idx] = N[idx] + 1



In [9]:
## TODO plot the N matrix 
## AIM: to show (a, b) -> c in a 2d landscape

In [42]:
N = (N + 1).float()
P = N / N.sum(2, keepdim=True) # use softmax over the third dim
P.shape

torch.Size([27, 27, 27])

### inference

In [48]:
for i in range(5):
    out = []
    idx = 0,0
    while True:
        p = P[idx]
        
        new = torch.multinomial(p, num_samples=1, replacement=True).item()
        
        idx = idx[1],new
        if new == 0:
            break
        out.append(itos[new])
        
    print(''.join(out))

in
kron
ni
kelin
hle


### loss

In [50]:
n = 0 
loss = 0
for word in words[:1]:
    w = ['.', '.'] + list(word) + ['.']

    for ch1, ch2, ch3 in zip(w, w[1:], w[2:]):
        idx = (stoi[ch1], stoi[ch2], stoi[ch3])
        loss += torch.log(P[idx])
        n += 1

nll = -loss/n
print(f"{nll=}")

nll=tensor(2.0956)


##  trigram linear

In [53]:
import torch.nn.functional as F

In [113]:
X1, X2, Y = [], [], []
for word in words:
    w = ['.', '.'] + list(word) + ['.']

    for ch1, ch2, ch3 in zip(w, w[1:], w[2:]):
        X1.append(stoi[ch1])
        X2.append(stoi[ch2]) 
        Y.append(stoi[ch3])
      
assert len(X1) == len(X2) == len(Y)


In [108]:
x1_enc = F.one_hot(torch.tensor(X1), num_classes=27).float()
x2_enc = F.one_hot(torch.tensor(X2), num_classes=27).float()

assert x1_enc.shape  == x2_enc.shape

In [109]:
x1_enc[:3]

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [110]:
X1[:3]

[0, 0, 5]

In [115]:
nelem = x1_enc.shape[0]

In [138]:
W1 = torch.randn((27,27), requires_grad=True)
W2 = torch.randn((27,27), requires_grad=True)
assert W1.grad == W2.grad == None

In [118]:
count = torch.exp(x1_enc @ W1) + torch.exp(x2_enc @ W2)
count

tensor([[3.6539, 1.7893, 1.4137,  ..., 3.9771, 1.9272, 2.7941],
        [4.9465, 1.4733, 1.9900,  ..., 2.6424, 6.6975, 2.7044],
        [2.3388, 7.7381, 2.4071,  ..., 2.2920, 3.7197, 1.0585],
        ...,
        [1.7059, 1.5831, 1.2509,  ..., 0.7952, 1.7543, 1.4137],
        [4.6943, 3.6816, 2.8614,  ..., 4.1815, 2.7621, 2.1636],
        [1.2592, 1.2549, 0.6223,  ..., 0.3890, 3.9024, 1.4291]],
       grad_fn=<AddBackward0>)

In [120]:
prob = count / count.sum(1, keepdims=True)

In [121]:
loss = -prob[torch.arange(nelem), Y].log().mean() + 0.01 * (W1**2 + W2**2).mean()
loss

tensor(3.5810, grad_fn=<AddBackward0>)

In [139]:
for k in range(1000):
    counts = torch.exp(x1_enc @ W1) + torch.exp(x2_enc @ W2)
    prob = counts / counts.sum(1, keepdim=True)

    loss = -prob[torch.arange(nelem), Y].log().mean() + 0.01 * (W1**2).mean() + 0.01 * (W2**2).mean()

    
    if  k%100 == 0:
        print(loss.item())
    W1.grad, W2.grad = None, None
    
    loss.backward()

    W1.data += -50*2 * W1.grad
    W2.data += -50*2 * W2.grad
    

3.5741629600524902
2.4722166061401367
2.448169469833374
2.441180944442749
2.4372003078460693
2.4344844818115234
2.433318614959717
2.4328253269195557
2.432562828063965
2.4324092864990234


In [159]:
for i in range(5):
    out = []
    idx = 0,0
    while True:
        X1_enc = F.one_hot(torch.tensor([idx[0]]), num_classes=27).float()
        X2_enc = F.one_hot(torch.tensor([idx[1]]), num_classes=27).float()

        counts = torch.exp(X1_enc @ W1) + torch.exp(X2_enc @ W2)

        prob = counts / counts.sum(1, keepdim=True)
        
        new = torch.multinomial(prob, num_samples=1, replacement=True).item()
        
        idx = idx[1],new
        if new == 0:
            break
        out.append(itos[new])
        
    print(''.join(out))

lenie
tin
an
ah
kahalo
